In [1]:
import pandas as pd
import re
import time
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from selenium import webdriver
from textacy.preprocess import preprocess_text
import nltk

from bs4 import BeautifulSoup as bs

In [2]:
start = time.time()

wd = webdriver.Chrome()

b_summary = []
b_detail = []
b_date = []
b_summary_clean = []
b_detail_clean = []

for i in range(1,5):
    print 'scrapping page : ', i
    url = 'http://gninethree.com/page/' + str(i) + '/'   

    try:
        wd.get(url)
    except:
        continue
    response = wd.page_source
    soup = bs(response, 'html.parser')
    
    try:
        blogs = soup.find_all('h2', {'class' : 'title'})
    except:
        continue
        
    for summary in blogs:
        try:
            href = summary.find('a').get('href')
            print 'href : ', href
        except:
            continue            
        
        try:
            wd.get(href)
        except:
            continue
        
        r = wd.page_source
        s = bs(r, 'html.parser')

        # find date
        try:
            date = s.find('span', {'class' : 'meta_date'}).text
        except:
            continue
            
        # find detail
        try:
            y = s.find('div', {'class' : "entry"}).find_all('p')
        except:
            continue
        doc = []
        for p in y:
            if p.text:
                doc.append(p.text)
        d = ' '.join(doc)

        b_summary.append(summary.text)
        b_detail.append(d)
        b_date.append(date)
        
        # clean summary and detail 
        clean_summary = preprocess_text(summary.text, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        clean_detail = preprocess_text(d, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        b_summary_clean.append(clean_summary)
        b_detail_clean.append(clean_detail)
        
        print 'cleaned detail snippet : ', clean_detail[:75]
        
end = time.time()
exe_time = (end - start)/60
print 'Time taken :',(exe_time),' minutes'

print len(b_date)

scrapping page :  1
href :  http://gninethree.com/2017/10/24/ps-cafe-opens-one-fullerton/
cleaned detail snippet :  pscafe has opened an outlet at one fullerton this wellloved caferestaurant 
href :  http://gninethree.com/2017/10/17/travel-mutekiya-ramen-%e7%84%a1%e6%95%b5%e5%ae%b6-in-tokyo/
cleaned detail snippet :  when my colleague first told read incessantly nagged at me to visit this ra
href :  http://gninethree.com/2017/09/03/the-bird-southern-table-bar-marina-bay-sands/
cleaned detail snippet :  another franchise has opened in singapore and this time it is one that is s
href :  http://gninethree.com/2017/08/27/weekend-brunch-don-ho-social-kitchen-bar/
cleaned detail snippet :  don ho social kitchen bar opened earlier this year as the newest kid on the
href :  http://gninethree.com/2017/08/08/magnum-opens-1st-magnum-pleasure-store-in-singapore-ion-orchard/
cleaned detail snippet :  if you are one of those who would eagerly anticipate whenever magnum launch
scrapping page :  2
hre

In [3]:
# remove stop words from cleaned summary and cleaned detail and lemmatize them

stoplist = set(stopwords.words('english') + \
               ['number',"singapore", "food", "im", "street", 
                'porn','watch', 'video', 'centre', '...',
                'eat', 'day', 'time', 'cdataadsbygoogle', 'windowadsbygoogle',
                'wa', 'ha', 'come', 'place', 'dish', 'bring', 'think', 'quite','located',
                'month', 'went', 'probably', 'am', 'pm', 'say', 'said','including','year','item',
                'youre', 'sure', 'dont', 'came','really', 'got', 'thing', 'address', 'photo',
                'credit', 'opening', 'hour'] \
                 + list(ENGLISH_STOP_WORDS))

In [4]:
start = time.time()

summary_tokens = []
for item in b_summary_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    summary_tokens.append(t)
    
detail_tokens = []
for item in b_detail_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    detail_tokens.append(t)   

    
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'
    
print len(summary_tokens)
print len(detail_tokens)

Time taken : 0.134683847427  seconds
19
19


In [5]:
start = time.time()

summary_lem =[]
wnl = WordNetLemmatizer()
for item in summary_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    summary_lem.append(t) 
    
detail_lem =[]
wnl = WordNetLemmatizer()
for item in detail_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    detail_lem.append(t)     
   
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'

print len(summary_lem)
print len(detail_lem)

Time taken : 3.19562101364  seconds
19
19


In [6]:
# create final list to fit in dataframe
flist = []
for i in range(len(b_summary)):
    f = [b_summary[i], b_detail[i], b_date[i], summary_lem[i], detail_lem[i]]
    flist.append(f)

In [7]:
df = pd.DataFrame(flist, columns = ['Title', 'Details', 'Date', 'Processed Summary', 'Processed Detail'])  
df.tail()

,Title,Details,Date,Processed Summary,Processed Detail
14,Bee’s Knees Cafe by The Garage @ Singapore Bot...,From the same people who opened One Rochester ...,"January 30, 2017",bee knee cafe garage botanic garden cluny park...,people opened rochester group numberaltitude c...
15,CNY ’17: Oriental Celebration in Luxurious Par...,The Lunar New Year is less than a week away an...,"January 24, 2017",cny oriental celebration luxurious paradise ca...,lunar new week away looking restaurant celebra...
16,"Tai Cheong Bakery @ Holland Village, 31 Lorong...","If you have not heard of “Tai Cheong Bakery”, ...","January 7, 2017",tai cheong bakery holland village lorong liput,heard tai cheong bakery friend missing deemed ...
17,"Ah Hua Kelong Seafood Restaurant @ Pasarbella,...",Singaporeans love their seafood and having sea...,"January 1, 2017",ah hua kelong seafood restaurant pasarbella gr...,singaporean love seafood seafood bag fast comm...
18,Christmas ’16: Clinton Street Baking Co. Chris...,Christmas is just around the corner and if you...,"December 23, 2016",christmas clinton baking christmas menu special,christmas corner right celebrate special occas...


In [8]:
len(df)

19

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
len(df)

19

In [11]:
x = df.iloc[3,1]
print x

DON HO- Social Kitchen & Bar opened earlier this year as the newest kid on the block along Keong Saik road. Sassy and stylish with just a little bit of funk, this tropical-themed restaurant served up Australian-fusion food with a touch of Middle-east in the evenings paired with vibrant eclectic cocktails. Not too long ago however, DON HO also rolled out their Brunch menu which is now available only on weekends. Decidedly modest, the menu features ten dishes which are sufficient for a complete weekend brunch experience, island-style. We started with two smoothies, the Flamingo Paradise $12 for me and Winter is Coming $12 for him (he is an avid Game of Thrones fan you see). The pink one which was mine could not be faulted for it was creamy, sweet and refreshing. Made up of strawberry, mango, banana and coconut milk, you probably can understand why I enjoyed it so much. His on the other hand, was the green one- made up of all things good and detoxifying. Kale, cinnamon, ginger, apricot an

In [12]:
# pickle dataframe
df.to_pickle('gninethree.pickle')

In [13]:
wd.quit()